In [5]:
import numpy as np
from math import exp, sqrt
import matplotlib.pyplot as plt
import BayesODE.Bayesian as bo
from BayesODE.Tests.root_gen import root_gen
from BayesODE.utils.utils import mvCond
from BayesODE.Kalman.kalman_initial_draw import kalman_initial_draw
from BayesODE.Kalman.kalman_ode_higher import kalman_ode_higher
from BayesODE.Kalman.higher_mvCond import higher_mvCond
from BayesODE.Tests.test_exp_integrate import cov_yy_ex
from BayesODE.Kalman.cov_car import cov_car
from BayesODE.Kalman.Old.kalman_filter import kalman_filter
from BayesODE.Kalman.Old.kalman_smooth import kalman_smooth
from BayesODE.Kalman.pykalman.filter_update_full import filter_update_full
from pykalman import standard as pks
import scipy as sc

In [6]:
from math import sin, cos
def chk_F(y_t, t):
    return sin(2*t) - y_t[0] #X^{2} = sin(2t) - X

def chk_exact_x(t):
    return (-3*cos(t) + 2*sin(t) - sin(2*t))/3

def chk_exact_x1(t):
    return (-2*cos(2*t) + 3*sin(t) + 2*cos(t))/3

def chk_exact_x2(t):
    return (4*sin(2*t) - 2*sin(t) + 3*cos(t))/3


# Test Against Pykalman

In [7]:
#Define parameters
N = 2
q = 2
p = q+2

delta_t = np.array([1/N])
r0 = 0.5
sigma = 0.001
roots = root_gen(r0, p) #Generate roots to draw x^{(3)}_0
a = np.array([0,0,1])
x0 = np.array([-1,0,0])
x0 = np.array([-1,0,chk_F(x0, 0)]) #Initial state

Y0 = kalman_initial_draw(roots, sigma, x0, p)
A, V = higher_mvCond(delta_t, roots, sigma) 
lam = np.zeros(p)
b = lam - A.dot(lam)
fun = chk_F

In [8]:

"""
.. module:: kalman_smooth

Backward pass algorithm to smooth a Kalman Filter estimate.
"""
import numpy as np

def kalman_smooth(A, mu_currs, Sigma_currs, mu_preds, Sigma_preds):
    """Backward pass to smooth a Kalman Filter estimate.
    Perform smoothing to estimate the state at time :math:`t`
    give an observation at time :math:`[0...N]` for :math: `t=0:N`.  This
    method is useful if one wants to track an object with streaming
    observations.

    Parameters
    ----------
    A : [n_dim_state, n_dim_state] :obj:`numpy.ndarray`
        state transition matrix from time t to t+1.
    mu_currs : [n_timesteps, n_dim_state] :obj:`numpy.ndarray`
        mean estimate for state at time t given observations from times
        [0...t].
    Sigma_currs : [n_timesteps, n_dim_state, n_dim_state] :obj:`numpy.ndarray`
        covariance of estimate for state at time t given observations from
        times [0...t]
    mu_preds : [n_timesteps, n_dim_state] :obj:`numpy.ndarray`
        mean of hidden state at time t+1 given observations from 
        times [0...t+1]
    Sigma_preds : [n_timesteps, n_dim_state, n_dim_state] :obj:`numpy.ndarray`
        covariance of hidden state at time t+1 given observations from 
        times [0...t+1]

    Returns
    -------
    Y_tt : [n_timesteps, n_dim_state] :obj:`numpy.ndarray`
        samples at time t given observations from 
        times [0...N] for t=0:N.
    mu_smooth : [n_timesteps, n_dim_state] :obj:`numpy.ndarray`
        mean of hidden state at time t given observations from 
        times [0...N] for t=0:N.
    Sigma_pred : [n_timesteps, n_dim_state, n_dim_state] :obj:`numpy.ndarray`
        covariance of hidden state at time t given observations from 
        times [0...N] for t=0:N.
    """
    n_timesteps, n_dim_state = mu_currs.shape
    mu_smooth = np.zeros((n_timesteps, n_dim_state))
    Sigma_smooth = np.zeros((n_timesteps, n_dim_state, n_dim_state))
    h_tt = np.zeros((n_timesteps, n_dim_state))
    W_tt = np.zeros((n_timesteps, n_dim_state, n_dim_state))
    Y_tt = np.zeros((n_timesteps, n_dim_state))
    
    #Base Case Smoothing Sampler
    h_tt[-1] = mu_smooth[-1] = mu_currs[-1] #\mu_{t|N} = \mu_{N|N} for t=N
    W_tt[-1] = Sigma_smooth[-1] = Sigma_currs[-1] #\Sigma_{t|N} = \Sigma{N|N} for t=N
    Y_tt[-1] = np.random.multivariate_normal(mu_smooth[-1], Sigma_smooth[-1])
    
    #Iterative Step
    for t in reversed(range(n_timesteps-1)):
        #B = \Sigma{t|t}A'_{t+1}
        B = Sigma_currs[t].dot(A.T)
        
        #\tilde{B} = \Sigma{t|t}A'_{t+1} \Sigma_{t+1|t}^{-1}
        B_tilde = B.dot(np.linalg.pinv(Sigma_preds[t+1]))
        
        #h_t = \mu_{t|t} + B \mu_{t+1|t}^{-1} (Z_{t+1} - \mu{t+1|t})
        h_tt[t] = mu_currs[t] + B_tilde.dot(Y_tt[t+1] - mu_preds[t+1])
        
        #W_t = \Sigma{t|t} - B \Sigma{t+1|t}^{-1} B'
        W_tt[t] = Sigma_currs[t] - B_tilde.dot(B.T)
        
        #Y_t ~ N{h_t, W_t}
        Y_tt[t] = np.random.multivariate_normal(h_tt[t], W_tt[t])
        
        #\mu_{t|N} = \mu_{t|t} + \tilde{B} (\mu_{t+1|N} - \mu_{t+1|t})
        mu_smooth[t] = mu_currs[t] + B_tilde.dot(mu_smooth[t+1] - mu_preds[t+1])
        
        #\Sigma_{t+1|N} = \Sigma_{t|t} + \tilde{B} (\Sigma_{t+1|N} - \Sigma_{t+1|t}) \tilde{B}'
        Sigma_smooth[t] = Sigma_currs[t] + np.linalg.multi_dot([B_tilde, (Sigma_smooth[t+1] - Sigma_preds[t+1]), B_tilde.T])
        
    return (h_tt, W_tt, Y_tt, mu_smooth, Sigma_smooth)

In [9]:
n_dim_obs = 1
n_dim_state = len(Y0)
n_timesteps = N+1

# allocate memory
us = np.zeros((n_timesteps,n_dim_obs)) 

# var(us_n | y_n), to be determined during the interrogation process
sig2 = np.zeros((n_timesteps, n_dim_obs, n_dim_obs))

# solution process
# forward mean and variance.
mu = np.zeros((n_timesteps, n_dim_state)) # E[y_n | us_0:n]
# var(y_n | us_0:n)
Sigma = np.zeros((n_timesteps, n_dim_state, n_dim_state))

#a padde with 0s
p = len(b)
q = len(a) - 1
a_star = np.pad(a, (0, p-q-1), 'constant', constant_values=(0,0))

# argumgents for kalman_filter and kalman_smooth
D = np.array([a_star])
e = np.array([0.])
F = sig2
mu_currs = mu
Sigma_currs = Sigma
mu_preds = np.zeros((n_timesteps, n_dim_state))
Sigma_preds = np.zeros((n_timesteps, n_dim_state, n_dim_state))

# arguments to use low-level pykalman functions
observations = us
observation_matrix = np.array([a_star])
observation_offset = np.array([0.])
observation_covariances = sig2 # multidimensional
transition_matrix = A
transition_offset = b
transition_covariance = V # single dimensional
filtered_state_means = mu
filtered_state_covariances = Sigma
predicted_state_means = np.zeros((n_timesteps, n_dim_state))
predicted_state_covariances = np.zeros((n_timesteps, n_dim_state, n_dim_state))

# initialize things
mu[0] = Y0
us[0] = Y0.dot(a_star)
mu_preds[0] = mu[0]
predicted_state_means[0] = mu[0]
Sigma_preds[0] = Sigma[0]
predicted_state_covariances[0] = Sigma[0]

# forward pass: merging pks._filter to accommodate multiple
# observation_covariances
# calculate mu_tt = E[y_t | us_0:t-1] and
# Sigma_tt = var(y_t | us_0:t-1)

for t in range(N):
    mu_tt = np.dot(A, mu[t]) + b
    Sigma_tt = np.linalg.multi_dot([A, Sigma[t], A.T]) + V #A*Sigma[t]*A.T + V 
    sig2[t+1] = np.linalg.multi_dot([a_star, Sigma_tt, a_star.T]) # new observation_covariance
    I_tt = np.random.multivariate_normal(np.zeros(p), np.eye(p))
    D_tt = np.linalg.cholesky(np.absolute(Sigma_tt, where=np.eye(p, dtype=bool)))
    Yt1 = mu_tt + D_tt.dot(I_tt) #Y_{t+1} ~ p(Y_{t+1} | Y_t)
    us[t+1] = fun(Yt1,(t+1)/N) #new observation (u_{t+1})

    (mu_preds[t+1], Sigma_preds[t+1], mu_currs[t+1], Sigma_currs[t+1]) = (
        kalman_filter(mu_curr = mu_currs[t],
                    Sigma_curr = Sigma_currs[t],
                    u_star = us[t+1],
                    A = A,
                    b = b,
                    V = V,
                    D = D,
                    e = e,
                    F = F[t+1]
                    )
    )
    (predicted_state_means[t+1], predicted_state_covariances[t+1],
         _, filtered_state_means[t+1],
         filtered_state_covariances[t+1]) = (
             filter_update_full(filtered_state_mean = filtered_state_means[t],
                                filtered_state_covariance = filtered_state_covariances[t],
                                observation = observations[t+1],
                                transition_matrix = transition_matrix,
                                transition_offset = transition_offset,
                                transition_covariance = transition_covariance,
                                observation_matrix = observation_matrix,
                                observation_offset = observation_offset,
                                observation_covariance = observation_covariances[t+1])
         )
# backward pass
(htt,Wtt, Y_tt, mu_smooth, Sigma_smooth) = (
    kalman_smooth(
        A = A, 
        mu_currs = mu_currs,
        Sigma_currs = Sigma_currs, 
        mu_preds = mu_preds,
        Sigma_preds = Sigma_preds
    )
)
# backward pass
(smoothed_state_means, smoothed_state_covariances, _) = (
    pks._smooth(
        transition_matrix, filtered_state_means,
        filtered_state_covariances, predicted_state_means,
        predicted_state_covariances
    )
)

Yn_mean = mu_smooth
Yn_var = Sigma_smooth

Yn_pykalman_mean = smoothed_state_means
Yn_pykalman_var = smoothed_state_covariances

In [10]:
print("Means are equal:{}".format(np.allclose(Yn_mean, Yn_pykalman_mean)))
print("Covariances are equal:{}".format(np.allclose(Yn_var, Yn_pykalman_var)))

Means are equal:True
Covariances are equal:True


# MVGSS

To differentiate the notation used in the general model and the state-space model. I will use double letter for the general model. For example, we have

$AA_n =\begin{bmatrix} A_n & 0 \\ D_nA_n & 0 \end{bmatrix}$, 
$bb_n = \begin{bmatrix} b_n \\ D_nb_n + e_n \end{bmatrix}$, and 
$CC_n = \begin{bmatrix} C_n & 0 \\ D_nC_n & F_n \end{bmatrix}$.

We also have 

$bb_0 = \begin{bmatrix} b_0 \\ D_0b_0 \end{bmatrix}$ and 
$CC_0 = \begin{bmatrix} C_0 & 0 \\ D_0C_0 & F_0 \end{bmatrix}$

where 

$A_n = A$, $b_0 = \lambda$, $b_n = \lambda - A \lambda$, $D_n = a_\star'$, $e_n = 0$, $F_0 = 0$ and $F_n^2 = \sigma_n^2 = a_\star'\Sigma_{n|n-1}a_\star$.

In [11]:
def ss2gss(AA, bb, CC, DD, ee, FF):
    p = bb.shape[0]
    len_y = bb.shape[0] + ee.shape[0]
    N = ee.shape[1] - 1
    
    Aa = np.zeros((len_y, len_y, N+1))
    bB = np.zeros((len_y, N+1))
    Cc = np.zeros((len_y, len_y, N+1))
    for i in range(N+1):
        #Aa
        Aan = np.zeros((len_y, len_y))
        Aan[0:p,0:p] = AA[:,:,i]
        Aan[p,0:p] = DD[:,i].dot(AA[:,:,i])
        Aa[:,:,i] = Aan
        
        #bB
        bBn = np.zeros(len_y)
        bBn[0:p] = bb[:,i]
        bBn[p] = DD[:,i].dot(bb[:,i]) + ee[:,i]
        bB[:,i]  = bBn
        
        #Cc
        Ccn = np.zeros((len_y, len_y))
        Ccn[0:p,0:p] = CC[:,:,i]
        Ccn[p,0:p] = DD[:,i].dot(CC[:,:,i])
        Ccn[p,p] = FF[:,i]
        Cc[:,:,i] = Ccn
        
    Aa[:,:,0] = np.nan
    return Aa, bB, Cc


In [12]:
C = np.linalg.cholesky(V)
D = np.pad(a, (0, p-q-1), 'constant', constant_values=(0,0)) #a_star

AA = np.dstack([A]*(N+1))
bb = np.repeat(b[:,np.newaxis],N+1,axis=1)
CC = np.dstack([C]*(N+1))
DD = np.repeat(D[:,np.newaxis],N+1,axis=1)
ee = np.zeros((1, N+1))
FF = np.zeros((1, N+1))

AA[:,:,0] = np.nan
bb[:,0] = Y0
V_inf = cov_car([], roots, sigma, v_infinity=True) 
#CC[:,:,0] = np.linalg.cholesky(V_inf)
CC[:,:,0] = np.zeros((CC.shape[0], CC.shape[0]))

for i in range(N+1):
    FF[:,i] = np.sqrt(np.linalg.multi_dot([D.T, Sigma_preds[i], D]))


In [13]:
def mvGSS(AA, bb, CC):
    d,veclen = bb.shape
    N = veclen-1
    D = d*N + d
    An_m = np.zeros((d,d,N+1,N+1))
    for n in range(N+1):
        for m in range(N+1):
            if m>n:
                An_m[:,:,n,m] = np.eye(d)
            elif n==m:
                An_m[:,:,n,m] = AA[:,:,n]
            else:
                diff = n-m
                A_diff = AA[:,:,m]
                for i in range(diff):
                    A_diff = np.matmul(AA[:,:,m+i+1],A_diff)
                An_m[:,:,n,m] = A_diff
    L = np.zeros((D,D))
    mean_Y = np.zeros(D)
    for n in range(N+1):
        for m in range(n,N+1):
            if n == N:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(np.eye(d), CC[:,:,n])
            else:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(An_m[:,:,m,n+1], CC[:,:,n])
        for l in range(n):
            mean_Y[n*d:n*d+d] = mean_Y[n*d:n*d+d] + An_m[:,:,n,l+1].dot(bb[:,l])
    LL = L.dot(L.T)
    var_Y = LL
    return An_m, L, mean_Y, var_Y

In [14]:
Aa, bB, Cc = ss2gss(AA,bb,CC,DD,ee,FF)

In [15]:
An_m, L, mean_Y, var_Y = mvGSS(Aa,bB,Cc)

In [16]:
#Check Y_0
var_YY_0 = np.matmul(Cc[:,:,0], Cc[:,:,0].T)
np.allclose(var_Y[0:5, 0:5],var_YY_0)

True

In [17]:
#Check Y_1
var_YY_1 = np.linalg.multi_dot([Aa[:,:,1], var_YY_0, Aa[:,:,1].T]) + np.matmul(Cc[:,:,1],Cc[:,:,1].T)
np.allclose(var_Y[5:10, 5:10],var_YY_1)

True

In [18]:
#Check Y_2
var_YY_2 = np.linalg.multi_dot([Aa[:,:,2], var_YY_1, Aa[:,:,2].T]) + np.matmul(Cc[:,:,2],Cc[:,:,2].T)
np.allclose(var_Y[10:15, 10:15],var_YY_2)

True

### Kalman Filter Sanity Check

In [19]:
#Kalman Filter p(Z_2|X_{0:2})
icond = np.array([False, False, False, False, True]*2 + [False, False, False, False, False]*(N-2))
R,s,T = mvCond(mean_Y[5:], var_Y[5:,5:], icond)

In [20]:
np.allclose(R.dot(us[1:3]).flatten()[4:] + s[4:], mu_currs[2])

True

In [21]:
np.allclose(Sigma_currs[2], T[4:8,4:8])

True

### Kalman Smoothing Sampler

In [22]:
#Kalman Filter p(Z_1|X_{0:2}, Y_2)
icond = np.array([False, False, False, False, True] + [True, True, True, True, True])
R,s,T = mvCond(mean_Y[5:], var_Y[5:,5:], icond)

In [23]:
Smooth_obvs = np.append(np.append(us[1],Y_tt[2]),us[2])

In [24]:
np.allclose(htt[1], R.dot(Smooth_obvs).flatten() + s)

True

In [25]:
np.allclose(Wtt[1],T)

True